In [8]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

In [9]:
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([hard_womens,hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()


In [10]:
# Load the dataset
df = temp[(temp['Sex']!='Wobmens')&(temp['Fav_Odds']<2)&(temp['Fav_Odds']>1.4)]


# Feature selection and preprocessing
features = ['fav_percent', 'dog_percent', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 
            'fav_rank', 'dog_rank', 'dog_rank_high', 'fav_rank_high','Fav_Top100','Dog_Top100','Dog_Win_Count','Fav_Loss_Count']

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop the 'Winner_Odds' column
df = df.drop(columns=['Winner_Odds'])


target = 'Fav_Win'

X = df[features]  # Features for training
y = df[target]  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42,shuffle=False)

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Add predictions and actuals to the test set DataFrame
test_with_predictions = X_test.copy()
test_with_predictions['Actual'] = y_test
test_with_predictions['Predicted'] = y_pred

# Add Fav_Odds and Dog_Odds back to the test set for profit calculation
test_with_predictions = test_with_predictions.merge(
    df[['Date','Fav','Dog','Fav_Odds', 'Dog_Odds']], left_index=True, right_index=True
)

# Define the stake amount
stake = 100  # Example stake amount

# Profit calculation
def calculate_profit(row):
    if row['Predicted'] == 1:  # If the model predicts the favorite wins
        if row['Actual'] == 1:  # If the prediction is correct
            return stake * (row['Fav_Odds'] - 1)  # Profit based on favorite odds
        else:
            return -stake  # Loss of the stake
    else:  # If the model predicts the underdog wins
        if row['Actual'] == 0:  # If the prediction is correct
            return stake * (row['Dog_Odds'] - 1)  # Profit based on underdog odds
        else:
            return -stake  # Loss of the stake

test_with_predictions['Profit'] = test_with_predictions.apply(calculate_profit, axis=1)

# Save the output DataFrame to clipboard
fav_predictions=test_with_predictions[test_with_predictions['Predicted']==1].copy()
# Calculate overall profit
fav_profit = fav_predictions['Profit'].sum()
print(f"Fav Profit: ${fav_profit:.2f}",len(fav_predictions))

dog_predictions=test_with_predictions[test_with_predictions['Predicted']==0].copy()
# Calculate overall profit
dog_profit = dog_predictions['Profit'].sum()
print(f"Dog Profit: ${dog_profit:.2f}", len(dog_predictions))
dog_predictions.to_clipboard()

Accuracy: 0.5228215767634855
              precision    recall  f1-score   support

       False       0.46      0.70      0.56       103
        True       0.64      0.39      0.48       138

    accuracy                           0.52       241
   macro avg       0.55      0.55      0.52       241
weighted avg       0.56      0.52      0.51       241

Fav Profit: $264.00 85
Dog Profit: $488.00 156


In [12]:
import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
dog_predictions['Fav_Odds_Band'] = pd.cut(dog_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
dog_predictions['Correct_Prediction'] = dog_predictions['Actual'] == dog_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_dog = dog_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_dog)



import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
fav_predictions['Fav_Odds_Band'] = pd.cut(fav_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
fav_predictions['Correct_Prediction'] = fav_predictions['Actual'] == fav_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_fav = fav_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_fav)

  Fav_Odds_Band  Total_Profit  Record_Count  Winning_Percentage
0    1.0 to 1.1           0.0             0            0.000000
1    1.1 to 1.2           0.0             0            0.000000
2    1.2 to 1.3           0.0             0            0.000000
3    1.3 to 1.4           0.0             0            0.000000
4    1.4 to 1.5        1166.0            43           48.837209
5    1.5 to 1.6       -1086.0            34           29.411765
6    1.6 to 1.7        -823.0            38           36.842105
7    1.7 to 1.8         909.0            27           66.666667
8    1.8 to 1.9         322.0            14           64.285714
9    1.9 to 2.0           0.0             0            0.000000
  Fav_Odds_Band  Total_Profit  Record_Count  Winning_Percentage
0    1.0 to 1.1           0.0             0            0.000000
1    1.1 to 1.2           0.0             0            0.000000
2    1.2 to 1.3           0.0             0            0.000000
3    1.3 to 1.4           0.0           

In [13]:
today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.4]
today_features=today[features]
today['Predicted']=model.predict(today_features)
probabilities = model.predict_proba(today_features)

today['Predicted_Prob'] = probabilities[:, 0]  # Column 1 corresponds to the positive class
today=today[today['Predicted']==False][['Time','Dog','Predicted','Predicted_Prob','Dog_Odds','Fav_Odds']]
today = today[today['Dog_Odds'] >1]
today = today[today['Predicted_Prob']>0.65]
today



,Time,Dog,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds
6,12:35,Christian Langmo,False,0.66,2.21,1.61
8,14:10,Maxime Janvier,False,0.75,2.36,1.54
11,14:30,Sara Saito,False,0.80,1.98,1.76
13,16:00,Rebecca Marino,False,0.67,1.95,1.79
18,19:00,Nicolas Alvarez Varona,False,0.68,1.92,1.80
24,20:30,Lukas Pokorny,False,0.67,2.52,1.45


In [14]:
today[today['Predicted']==True][['Time','Fav','Predicted','Fav_Odds']]

KeyError: "['Fav'] not in index"

In [ ]:
#today['Date']='2025-01-08'
#today.to_sql('DailyPredictions',con=devengine,index=False,if_exists='append')

